# Data generation: using python to sweep over methods and parameters

In this notebook, we illustrate how to use python to generate and run a list of commands. In this example, we generate a list of QIIME 1.9.0 ``assign_taxonomy.py`` commands, though this workflow for command generation is generally very useful for performing *parameter sweeps* (i.e., exploration of sets of parameters for achieving a specific result for comparative purposes). 

## Environment preparation

In [16]:
from os import system, environ
from os.path import join, expandvars 
from joblib import Parallel, delayed
from glob import glob
from tax_credit.framework_functions import (recall_novel_taxa_dirs,
                                            parameter_sweep,
                                            move_results_to_repository)
from ggmap.snippets import *

ggmap is custome code from Stefan Janssen, download at https://github.com/sjanssen2/ggmap
Reading settings file '/home/sjanssen/.ggmaprc'


In [2]:
project_dir = "../.."
analysis_name= "novel-taxa-simulations"

results_dir = join('..', '..', 'novel-taxa-tmp')

## Preparing data set sweep

First, we're going to define the data sets that we'll sweep over. As the simulated novel taxa dataset names depend on how the database generation notebook was executed, we must define the variables used to create these datasets. If you modified any variables in that notebook, set these same variables below. If you did not, then do not modify.

recall_novel_taxa_dirs() generates a list of dataset_reference_combinations and a dictionary of reference_dbs mapped to each dataset, which we feed to parameter_sweep below.

In [43]:
iterations = 10
data_dir = join(project_dir, "data", analysis_name)
# databases is a list of names given as dictionary keys in the second
# cell of the database generation notebook. Just list the names here.
databases = ['B1-REF']

# Generate a list of input directories
(dataset_reference_combinations, reference_dbs) = recall_novel_taxa_dirs(data_dir, databases, iterations, min_level=1)

## Preparing the method/parameter combinations and generating commands

Now we set the methods and method-specific parameters that we want to sweep. Modify to sweep other methods. Note how method_parameters_combinations feeds method/parameter combinations to parameter_sweep() in the cell below.

### SEPP

In [56]:
method_parameters_combinations = {
    'sepp-paths': {'method': ['path'], 'reference_name': ['gg138']},
    'sepp-otus': {'method': ['otus'], 'reference_name': ['gg138']}
}

# this filepath needs to point to the directory in which the SEPP wrapper can be found
fp_wrapper_script = '../../../q2-fragment-insertion/tax-credit/'

#command_template = 'bash -c "source activate qiime1; source ./.bashrc; mkdir -p {0} ; assign_taxonomy.py -v -i {1} -o {0} -r {2} -t {3} -m {4} {5} --rdp_max_memory 16000"'
command_template = '%s/wrapper_sepp.py {1} {0}/query_tax_assignments.txt --novel_taxa_keep {2} --novel_taxa_level `echo "{1}" | sed "s#^.*-L##" | sed "s/-iter.*$//"` {5}' % fp_wrapper_script

command_template += ' --reference_alignment %s --reference_phylogeny %s' % (
    join(environ['CONDA_PREFIX'], 'share', 'fragment-insertion', 'ref', 'gg_13_5_ssu_align_99_pfiltered.fasta'),
    join(environ['CONDA_PREFIX'], 'share', 'fragment-insertion', 'ref', 'reference-gg-raxml-bl-rooted-relabelled.tre'))

commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.fasta', output_name='query_tax_assignments.txt')

As a sanity check, we can look at the first command that was generated and the number of commands generated.

In [57]:
for method in method_parameters_combinations:
    print(method)
    for command in commands:
        if '/'+method+'/' in command:
            print(command)
            break

sepp-otus
../../../q2-fragment-insertion/tax-credit//wrapper_sepp.py ../../data/novel-taxa-simulations/B1-REF-L6-iter5/query.fasta ../../novel-taxa-tmp/B1-REF-L6-iter5/B1-REF-L6-iter5/sepp-otus/otus:gg138/query_tax_assignments.txt --novel_taxa_keep ../../data/novel-taxa-simulations/B1-REF-L6-iter5/ref_seqs.fasta --novel_taxa_level `echo "../../data/novel-taxa-simulations/B1-REF-L6-iter5/query.fasta" | sed "s#^.*-L##" | sed "s/-iter.*$//"` --method otus --reference_name gg138 --reference_alignment /home/sjanssen/miniconda3/envs/qiime2-2018.6/share/fragment-insertion/ref/gg_13_5_ssu_align_99_pfiltered.fasta --reference_phylogeny /home/sjanssen/miniconda3/envs/qiime2-2018.6/share/fragment-insertion/ref/reference-gg-raxml-bl-rooted-relabelled.tre
sepp-paths


In [58]:
print(len(commands))

1


Finally, we run our commands.

In [8]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

In [60]:
for i, cmd in enumerate(sorted(commands)):
    cluster_run([cmd], 'taxcredit_sepp_%i' % i, "./dummy", environment='qiime2-2018.6', ppn=20,
                wait=False, dry=True)

source activate qiime2-2018.6; echo '../../../q2-fragment-insertion/tax-credit//wrapper_sepp.py ../../data/novel-taxa-simulations/B1-REF-L6-iter5/query.fasta ../../novel-taxa-tmp/B1-REF-L6-iter5/B1-REF-L6-iter5/sepp-otus/otus:gg138/query_tax_assignments.txt --novel_taxa_keep ../../data/novel-taxa-simulations/B1-REF-L6-iter5/ref_seqs.fasta --novel_taxa_level `echo "../../data/novel-taxa-simulations/B1-REF-L6-iter5/query.fasta" | sed "s#^.*-L##" | sed "s/-iter.*$//"` --method otus --reference_name gg138 --reference_alignment /home/sjanssen/miniconda3/envs/qiime2-2018.6/share/fragment-insertion/ref/gg_13_5_ssu_align_99_pfiltered.fasta --reference_phylogeny /home/sjanssen/miniconda3/envs/qiime2-2018.6/share/fragment-insertion/ref/reference-gg-raxml-bl-rooted-relabelled.tre' | /opt/torque-4.2.8/bin/qsub -d '/home/sjanssen/miniconda3/envs/qiime2-2018.6/src/tax-credit-data/ipynb/novel-taxa' -V -l walltime=4:00:00,nodes=1:ppn=20,pmem=8GB -N cr_taxcredit_sepp_0 -t 1-1 


### Assignment Using QIIME 1 or Command-Line Classifiers

Here we provide an example of taxonomy assignment using legacy ``QIIME 1`` classifiers executed on the command line. To accomplish this, we must first convert ``commands`` to a string, which we then pass to bash for execution. As ``QIIME 1`` is written in python-2, we must also activate a separate environment in which QIIME 1 [has been installed](http://qiime.org/install/install.html). If any environmental variables need to be set (in this example, the [RDP_JAR_PATH](http://qiime.org/install/alternative.html#rdp-jar-path-environment-variable)), we must also source the .bashrc file.

In [4]:
method_parameters_combinations = { # probabalistic classifiers
              'rdp': {'confidence': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5,
                                     0.6, 0.7, 0.8, 0.9, 1.0]},
              
              # global alignment classifiers
              'uclust': {'min_consensus_fraction': [0.51, 0.76, 1.0], 
                         'similarity': [0.8, 0.9],
                         'uclust_max_accepts': [1, 3, 5]},
             
              # local alignment classifiers
              'sortmerna': {'sortmerna_e_value': [1.0],
                            'min_consensus_fraction': [0.51, 0.76, 1.0], 
                            'similarity': [0.8, 0.9],
                            'sortmerna_best_N_alignments ': [1, 3, 5],
                            'sortmerna_coverage' : [0.8, 0.9]},
              'blast' : {'blast_e_value' : [0.0000000001, 0.001, 1, 1000]}
             }

Now enter the template of the command to sweep, and generate a list of commands with ``parameter_sweep()``.

Fields must adhere to following format:

                      {0} = output directory
                      {1} = input data
                      {2} = output destination
                      {3} = reference taxonomy
                      {4} = method name
                      {5} = other parameters

In [5]:
command_template = 'bash -c "source activate qiime1; source ./.bashrc; mkdir -p {0} ; assign_taxonomy.py -v -i {1} -o {0} -r {2} -t {3} -m {4} {5} --rdp_max_memory 16000"'

commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.fasta', output_name='query_tax_assignments.txt')


As a sanity check, we can look at the first command that was generated and the number of commands generated.

In [6]:
for method in method_parameters_combinations:
    print(method)
    for command in commands:
        if '/'+method+'/' in command:
            print(command)
            break

sortmerna
bash -c "source activate qiime1; source ./.bashrc; mkdir -p ../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/sortmerna/0.51:0.8:1:0.8:1.0 ; assign_taxonomy.py -v -i ../../data/novel-taxa-simulations/B1-REF-L6-iter0/query.fasta -o ../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/sortmerna/0.51:0.8:1:0.8:1.0 -r ../../data/novel-taxa-simulations/B1-REF-L6-iter0/ref_seqs.fasta -t ../../data/novel-taxa-simulations/B1-REF-L6-iter0/ref_taxa.tsv -m sortmerna --sortmerna_e_value 1.0 --min_consensus_fraction 0.51 --similarity 0.8 --sortmerna_best_N_alignments  1 --sortmerna_coverage 0.8 --rdp_max_memory 16000"
uclust
bash -c "source activate qiime1; source ./.bashrc; mkdir -p ../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/uclust/0.51:0.8:1 ; assign_taxonomy.py -v -i ../../data/novel-taxa-simulations/B1-REF-L6-iter0/query.fasta -o ../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/uclust/0.51:0.8:1 -r ../../data/novel-taxa-simulations/B1-REF-L6-iter0/ref_seqs.fasta -t ..

In [7]:
print(len(commands))

8280


Finally, we run our commands.

In [8]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

### BLAST+

In [9]:
(dataset_reference_combinations, reference_dbs) = recall_novel_taxa_dirs(
    data_dir, databases, iterations, ref_seqs='ref_seqs.qza', ref_taxa='ref_taxa.qza')

In [14]:
method_parameters_combinations = {
              'blast+' : {'p-evalue': [0.001],
                          'p-maxaccepts': [1, 10],
                          'p-perc-identity': [0.80, 0.97, 0.99],
                          'p-min-consensus': [0.51, 0.75, 0.99]}
             }

command_template = ("mkdir -p {0}; "
                    "qiime feature-classifier classify-consensus-blast --i-query {1} --o-classification "
                    "{0}/rep_seqs_tax_assignments.qza --i-reference-reads {2} --i-reference-taxonomy {3} {5}; "
                    "qiime tools export {0}/rep_seqs_tax_assignments.qza --output-dir {0}; "
                    "mv {0}/taxonomy.tsv {0}/query_tax_assignments.txt")
        

commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.qza', output_name='rep_seqs_tax_assignments.qza')

In [15]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

### VSEARCH

In [16]:
method_parameters_combinations = {
              'vsearch' : {'p-maxaccepts': [1, 10],
                           'p-perc-identity': [0.80, 0.90, 0.97, 0.99],
                           'p-min-consensus': [0.51, 0.99]}
             }

command_template = ("mkdir -p {0}; "
                    "qiime feature-classifier classify-consensus-vsearch --i-query {1} --o-classification "
                    "{0}/rep_seqs_tax_assignments.qza --i-reference-reads {2} --i-reference-taxonomy {3} {5}; "
                    "qiime tools export {0}/rep_seqs_tax_assignments.qza --output-dir {0}; "
                    "mv {0}/taxonomy.tsv {0}/query_tax_assignments.txt")
        
commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.qza', output_name='rep_seqs_tax_assignments.qza')

In [17]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

### scikit-learn

In [18]:
method_parameters_combinations = {
    'naive-bayes' : {'p-feat-ext--ngram-range': 
                     ['[4,4]', '[6,6]', '[8,8]', '[16,16]', '[32,32]',
                      '[7,7]', '[9,9]', '[10,10]', '[11,11]', 
                      '[12,12]', '[14,14]', '[18,18]'],
                     'p-classify--alpha': [0.001]},
    'naive-bayes-bespoke' : {'p-feat-ext--ngram-range': 
                             ['[4,4]', '[6,6]', '[8,8]', '[16,16]', '[32,32]',
                              '[7,7]', '[9,9]', '[10,10]', '[11,11]', 
                              '[12,12]', '[14,14]', '[18,18]'],
                             'p-classify--alpha': [0.001],
                             'p-classify--fit-prior': ['']}
}

command_template = ('mkdir -p "{0}"; '
                    'qiime feature-classifier fit-classifier-naive-bayes --o-classifier '
                    '"{0}/classifier.qza" --i-reference-reads {2} --i-reference-taxonomy {3} {5}; ')

confidences = [0.0, 0.5, 0.7, 0.9, 0.92, 0.94,
               0.96, 0.98, 1.0]
command_template += ''.join(
    'mkdir -p "{0}:' + str(c) + '"; '
    'qiime feature-classifier classify-sklearn '
    '--o-classification "{0}:' + str(c) + '/rep_seqs_tax_assignments.qza" '
    '--i-classifier "{0}/classifier.qza" '
    '--i-reads {1} --p-confidence ' + str(c) + '; '
    'qiime tools export "{0}:' + str(c) + '/rep_seqs_tax_assignments.qza" --output-dir "{0}:' + str(c) + '"; '
    'mv "{0}:' + str(c) + '/taxonomy.tsv" "{0}:' + str(c) + '/query_tax_assignments.txt"; 'for c in confidences)

command_template += 'rm "{0}/classifier.qza"; rmdir "{0}"'

commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.qza', output_name='rep_seqs_tax_assignments.qza')

In [19]:
print(len(commands), 'commands')
print('\n\n'.join(commands[0].split(';')))

2880 commands
mkdir -p "../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/naive-bayes-bespoke/0.001::[4,4]"

 qiime feature-classifier fit-classifier-naive-bayes --o-classifier "../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/naive-bayes-bespoke/0.001::[4,4]/classifier.qza" --i-reference-reads ../../data/novel-taxa-simulations/B1-REF-L6-iter0/ref_seqs.qza --i-reference-taxonomy ../../data/novel-taxa-simulations/B1-REF-L6-iter0/ref_taxa.qza --p-classify--alpha 0.001 --p-feat-ext--ngram-range [4,4] --p-classify--fit-prior 

 mkdir -p "../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/naive-bayes-bespoke/0.001::[4,4]:0.0"

 qiime feature-classifier classify-sklearn --o-classification "../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/naive-bayes-bespoke/0.001::[4,4]:0.0/rep_seqs_tax_assignments.qza" --i-classifier "../../novel-taxa-tmp/B1-REF-L6-iter0/B1-REF-L6-iter0/naive-bayes-bespoke/0.001::[4,4]/classifier.qza" --i-reads ../../data/novel-taxa-simulations/B1-REF-L6-iter0/

In [ ]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

## Move result files to repository

Add results to the tax-credit directory (e.g., to push these results to the repository or compare with other precomputed results in downstream analysis steps). The precomputed_results_dir path and methods_dirs glob below should not need to be changed unless if substantial changes were made to filepaths in the preceding cells.

In [21]:
precomputed_results_dir = join(project_dir, "data", "precomputed-results", analysis_name)
method_dirs = glob(join(results_dir, '*', '*', '*', '*'))
move_results_to_repository(method_dirs, precomputed_results_dir)